In [ ]:
!pip install dv
import cv2
import glob
import os
from tqdm import tqdm
import numpy as np
import dv
import copy
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

EVENT_PATH = "/content/drive/MyDrive/driving_346x260_noise_shot_dark_5p3Hz.txt"

OUTPUT = "/content/"

# Filtering parameters
REFRACTORY_PERIOD = 1000    # in us
NN_WINDOW = 8000           # in us

# DAVIS Camera's Dimension
HEIGHT = 260
WIDTH = 346

class Events(object):
    def __init__(self, num_events: int, width: int, height: int) -> np.ndarray:
        # events contains the following index:
        # t: the timestamp of the event.
        # x: the x position of the event.
        # y: the y position of the event.
        # p: the polarity of the event.
        self.events = np.zeros((num_events), dtype=[("t", np.uint64), ("x", np.uint16), ("y", np.uint16), ("p", np.bool_), ("s", np.bool_)])
        self.width = width
        self.height = height
        self.num_events = num_events

# Read event data from file
def process_text_file(filename: str) -> Events:
    with open(filename, 'r',buffering=4000000) as f:
        num_events = 0
        for _ in f:
            num_events += 1


    print("!!!!!",num_events)
    events = Events(num_events, WIDTH, HEIGHT)

    with open(filename, 'r',buffering=4000000) as f:
        for i, line in enumerate(tqdm(f)):
            event = line.split('\t')
            if len(event) != 5: print("!!!!!!!!!")
            assert len(event) == 5
            events.events[i]["x"], events.events[i]["y"],events.events[i]["t"], =  int(event[0])-1, int(event[1])-1,int(event[3])   #*1e6
            events.events[i]["p"] = True if int(event[2]) == 1 else False
            events.events[i]["s"] = int(event[4])
            #if(i==100): break

    return events



#  Spatiotemporal Correlation Filtering (STCF)
def Spatiotemporal_Correlation_Filter(event_array: Events, time_window: int=200, k: int=1):
    TP,TN,FP,FN=0,0,0,0
    max_x, max_y = event_array.width - 1, event_array.height - 1
    t0 = np.ones((event_array.height, event_array.width)) - time_window - 1
    x_prev, y_prev, p_prev= 0, 0, 0,
    valid_indices = np.ones(event_array.num_events, dtype=np.bool_)

    for i, e in tqdm(enumerate(event_array.events)): #tqdm: process bar;    // i is the index, e are thhe content(will go through each)
        count=0
        ts, x, y, p = e["t"], e["x"], e["y"], e["p"]

        if x_prev != x or y_prev != y or p_prev != p: #if install the first event, then not go into if () condition
            t0[y][x] = -time_window
            min_x_sub = max(0, x-1)
            max_x_sub = min(max_x, x+1)
            min_y_sub = max(0, y-1)
            max_y_sub = min(max_y, y+1)

            t0_temp = t0[min_y_sub:(max_y_sub+1), min_x_sub:(max_x_sub + 1)]
            for c in (ts-t0_temp.reshape(-1,1)):
                if c<= time_window: count+=1


            if count< k:
                valid_indices[i] = 0 #indixcate each event to tell nosie or signal
            if valid_indices[i]==1 and e["s"]==1: TP+=1
            if valid_indices[i]==0 and e["s"]==0: TN+=1
            if valid_indices[i]==1 and e["s"]==0: FP+=1
            if valid_indices[i]==0 and e["s"]==1: FN+=1

        t0[y][x], x_prev, y_prev, p_prev = ts, x, y, p #always update the timestamp
    print()
    print("TP",TP)
    print("TN",TN)
    print("FP",FP)
    print("FN",FN)
    with open('/content/drive/MyDrive/myfile.txt', 'a') as f:
        #f.writelines('!!k=0'+str(k)+'\n')
        s=str(TP)+'\t'+str(TN)+'\t'+str(FP)+'\t'+str(FN)+'\n'
        f.writelines(s)

    return event_array.events[valid_indices], np.count_nonzero(valid_indices == True)



if __name__ == "__main__":
    """
        Generate Video from events
    """
    current_events = process_text_file(EVENT_PATH)
    deepcopy = copy.deepcopy(current_events)
    for x in range(1):
      k=x+5
      print("!!!!! k=",k)
      with open('/content/drive/MyDrive/myfile.txt', 'a') as f:
        f.writelines('!!k=0'+str(k)+'\n')

    print()
    current_events = copy.deepcopy(deepcopy)
    print(2000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 2000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(3000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 3000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(4000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 4000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(5000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 5000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(6000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 6000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(7000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 7000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(8000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 8000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(9000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 9000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(10000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 10000)


    print()
    current_events = copy.deepcopy(deepcopy)
    print(12000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 12000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(13000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 13000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(14000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 14000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(15000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 15000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(16000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 16000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(17000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 17000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(18000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 18000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(19000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 19000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(20000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 20000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(22000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 22000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(23000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 23000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(24000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 24000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(25000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 25000)


    print()
    current_events = copy.deepcopy(deepcopy)
    print(26000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 26000)


    print()
    current_events = copy.deepcopy(deepcopy)
    print(27000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 27000)


    print()
    current_events = copy.deepcopy(deepcopy)
    print(28000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 28000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(29000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 29000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(30000)
    current_events.events, current_events.num_events =Spatiotemporal_Correlation_Filter(current_events, 30000)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(32000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 32000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(33000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 33000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(34000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 34000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(35000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 35000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(36000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 36000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(37000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 37000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(38000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 38000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(39000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 39000,k)

    print()
    current_events = copy.deepcopy(deepcopy)
    print(40000)
    current_events.events, current_events.num_events = Spatiotemporal_Correlation_Filter(current_events, 40000,k)




!!!!! 6677980


6677980it [00:47, 140305.09it/s]


!!!!! k= 5

2000


6677980it [08:36, 12936.00it/s]



TP 2259947
TN 2449721
FP 472660
FN 1495634

3000


...


TP 2690985
TN 2430728
FP 491653
FN 1064596

39000


6677980it [08:22, 13293.73it/s]



TP 2714975
TN 2416257
FP 506124
FN 1040606

40000


6677980it [08:18, 13383.54it/s]



TP 2737776
TN 2401869
FP 520512
FN 1017805
